# Introduction


This notebook shows how to run a technical indicator strategy against a live feed coming from the Binance crypto exchange. In this example we're not interested in actually back-testing of the strategy, just the signals it produces.

Whenever there is a new signal, whether it is a BUY or SELL, we get an notfication so we can react to it.

Roboquant includes a dedicated module for crypto support, including Binance that we'll use in this notebook. Since this module already has dependencies declared on the core module, we don't need to inlcude that one also.

In [ ]:
%use roboquant
Welcome()

In [ ]:
import org.roboquant.feeds.binance.*

# Setup
We now create, as usual, the components we want to use. We start with defining a strategy that uses a few technical analysis indicators to detects candlechart patterns and trends in moving averages. This might not be a very useful combination of indicators and just serves to demonstrate how to combine multiple ones.

In [ ]:
val strategy = TAStrategy(15)

strategy.buy {
    ta.cdlMorningStar(it) || ta.ema(it.close, 3) > ta.ema(it.close, 5)
}

strategy.sell {
    ta.cdl2Crows(it) && ta.ema(it.close, 3) < ta.ema(it.close, 5)
}

# Feed
Now we create an instance of the BinanceFeed and subscribe to pricebar events (candlesticks) of the the BTC/BUSD currency pair. It is also possible to subscribe to multiple currency pairs by adding additional parameters to the same *subscribePriceBar* method.

In [ ]:
val feed = BinanceFeed()
feed.subscribePriceBar(Pair("BTCBUSD","BUSD"))

# Policy

In this case we are only interested in the generated signals and not the actual testing. We can use the NotificationPolicy for this. This policy will send an email notification everytime it receives one or more signals from the strategy. But it will not generate any orders and thus the account stays unchanged. 

It is good practice not to use visible passwords in your notebook, so we rely here on that password being available through system or environment property. 

In [ ]:
val notifier = ConsoleNotifier()

// Can also use email as a notifier. However this needs to be enabled at GMail account otherwise username/password authentication not allowed.
// val notifier = EmailNotifier.usingGmail("username@gmail.com", "your_password", "to_email_address")

val policy = NotificationPolicy(notifier)

Because we won't be using USD as a base currency, we create a non-default Simbroker with an initial deposit of 100.000,- Binance USD (BUSD). We also define some metrics that will help afterwards to check some statistics:

1) ProgressMetric: overall progress of the run
2) PriceMetric: capture the recorded prices for an asset

In [ ]:
val broker = SimBroker.withDeposit(1_000_000.00, "BUSD")
val priceMetric = PriceMetric(feed.assets.first())
val roboquant =  Roboquant(strategy, ProgressMetric(), priceMetric, broker = broker)

# Live Test
All that remains is to start the run. We run it for 60 minutes, but you can run it of course for any duration.

In [ ]:
val timeFrame = TimeFrame.nextMinutes(60)
roboquant.run(feed, timeFrame)

# Results

In [ ]:
roboquant.logger.summary()

In [ ]:
PriceBarChart(priceMetric, priceMetric.asset)